<a href="https://colab.research.google.com/github/prayanshgupta129/CreditScoreGenerator/blob/main/score_wallets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import json
import zipfile
import io
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def load_and_preprocess_data(zip_file_path="aave_transactions.zip", json_file_name="user-wallet-transactions.json"):
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zf:
            with zf.open(json_file_name) as f:
                data = json.load(f)
        df = pd.DataFrame(data)
    except FileNotFoundError:
        print(f"Error: {zip_file_path} not found. Please download the file and place it in the same directory.")
        exit()
    except KeyError:
        print(f"Error: {json_file_name} not found inside {zip_file_path}. Please check the zip file content.")
        exit()
    except Exception as e:
        print(f"An error occurred while loading data: {e}")
        exit()
    if 'userWallet' in df.columns:
        df.rename(columns={'userWallet': 'wallet'}, inplace=True)
    else:
        print("Error: 'userWallet' column not found in the loaded data. Please check your JSON file structure.")
        print(f"Available columns: {df.columns.tolist()}")
        exit()

    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

    if 'actionData' not in df.columns:
        print("Error: 'actionData' column not found in the loaded data. Please check your JSON file structure.")
        print(f"Available columns: {df.columns.tolist()}")
        exit()

    action_data_df = df['actionData'].apply(pd.Series)

    required_action_data_keys = ['amount', 'assetPriceUSD']
    for key in required_action_data_keys:
        if key not in action_data_df.columns:
            print(f"Error: Required key '{key}' not found within 'actionData' dictionaries.")
            print(f"Available keys in actionData: {action_data_df.columns.tolist()}")
            exit()

    df['raw_amount'] = pd.to_numeric(action_data_df['amount'], errors='coerce')
    df['assetPriceUSD'] = pd.to_numeric(action_data_df['assetPriceUSD'], errors='coerce')

    df['amount'] = df['raw_amount'] * df['assetPriceUSD']

    df.dropna(subset=['amount', 'action'], inplace=True) # Ensure 'amount' (USD value) and 'action' are not NaN

    df.drop(columns=['raw_amount', 'assetPriceUSD', 'actionData', '_id', 'network', 'protocol', 'txHash', 'logId', 'blockNumber', 'assetSymbol', 'type', 'poolId', 'userId', '__v', 'createdAt', 'updatedAt'], errors='ignore', inplace=True)

    return df

def engineer_features(df):
    wallet_features = df.groupby('wallet').agg(
        total_transactions=('wallet', 'size'),
        unique_actions=('action', 'nunique'),
        first_tx_timestamp=('timestamp', 'min'),
        last_tx_timestamp=('timestamp', 'max'),
        total_deposited_value=('amount', lambda x: x[df.loc[x.index, 'action'] == 'deposit'].sum()),
        total_borrowed_value=('amount', lambda x: x[df.loc[x.index, 'action'] == 'borrow'].sum()),
        total_repaid_value=('amount', lambda x: x[df.loc[x.index, 'action'] == 'repay'].sum()),
        total_redeemed_value=('amount', lambda x: x[df.loc[x.index, 'action'] == 'redeemUnderlying'].sum()),
        liquidation_events_count=('action', lambda x: (x == 'liquidationCall').sum()),
        total_liquidated_value=('amount', lambda x: x[df.loc[x.index, 'action'] == 'liquidationCall'].sum())
    )

    wallet_features['activity_span_days'] = (wallet_features['last_tx_timestamp'] - wallet_features['first_tx_timestamp']).dt.days
    wallet_features['activity_span_days'] = wallet_features['activity_span_days'].replace(0, 0.001)

    wallet_features['avg_tx_per_day'] = wallet_features['total_transactions'] / wallet_features['activity_span_days']

    wallet_features['net_deposit_value'] = wallet_features['total_deposited_value'] - wallet_features['total_redeemed_value']
    wallet_features['net_borrow_value'] = wallet_features['total_borrowed_value'] - wallet_features['total_repaid_value']

    wallet_features['repay_to_borrow_ratio'] = wallet_features.apply(
        lambda row: row['total_repaid_value'] / row['total_borrowed_value'] if row['total_borrowed_value'] > 0 else (1.0 if row['total_borrowed_value'] == 0 else 0.0),
        axis=1
    )
    wallet_features['repay_to_borrow_ratio'] = wallet_features['repay_to_borrow_ratio'].clip(upper=1.0)

    wallet_features = wallet_features.drop(columns=['first_tx_timestamp', 'last_tx_timestamp'])

    wallet_features = wallet_features.fillna(0)

    return wallet_features

def assign_credit_scores(wallet_features):
    scores = pd.Series(0.0, index=wallet_features.index)

    scaler = MinMaxScaler()

    if wallet_features['total_transactions'].nunique() > 1:
        wallet_features['total_transactions_scaled'] = scaler.fit_transform(wallet_features[['total_transactions']])
    else:
        wallet_features['total_transactions_scaled'] = 0.5

    if wallet_features['activity_span_days'].nunique() > 1:
        wallet_features['activity_span_days_scaled'] = scaler.fit_transform(wallet_features[['activity_span_days']])
    else:
        wallet_features['activity_span_days_scaled'] = 0.5

    scores = 200 * (0.5 * wallet_features['total_transactions_scaled'] + 0.5 * wallet_features['activity_span_days_scaled'])

    scores = scores.apply(lambda x: max(x, 50))

    scores += 400 * wallet_features['repay_to_borrow_ratio']

    wallet_features['net_deposit_value_positive'] = wallet_features['net_deposit_value'].clip(lower=0)
    if wallet_features['net_deposit_value_positive'].max() > 0 and wallet_features['net_deposit_value_positive'].nunique() > 1:
        scaler.fit(wallet_features[['net_deposit_value_positive']])
        wallet_features['net_deposit_value_scaled'] = scaler.transform(wallet_features[['net_deposit_value_positive']])
        scores += 200 * wallet_features['net_deposit_value_scaled']
    else:
        wallet_features['net_deposit_value_scaled'] = 0

    if wallet_features['liquidation_events_count'].max() > 0 and wallet_features['liquidation_events_count'].nunique() > 1:
        scaler.fit(wallet_features[['liquidation_events_count']])
        wallet_features['liquidation_events_count_scaled'] = scaler.transform(wallet_features[['liquidation_events_count']])
        scores -= 150 * wallet_features['liquidation_events_count_scaled']
    else:
        wallet_features['liquidation_events_count_scaled'] = 0

    wallet_features['net_borrow_value_positive'] = wallet_features['net_borrow_value'].clip(lower=0)
    if wallet_features['net_borrow_value_positive'].max() > 0 and wallet_features['net_borrow_value_positive'].nunique() > 1:
        scaler.fit(wallet_features[['net_borrow_value_positive']])
        wallet_features['net_borrow_value_scaled'] = scaler.transform(wallet_features[['net_borrow_value_positive']])
        scores -= 50 * wallet_features['net_borrow_value_scaled']
    else:
        wallet_features['net_borrow_value_scaled'] = 0

    scores = scores.clip(lower=0, upper=1000).astype(int)

    return scores

def main():
    print("Loading and preprocessing data...")
    df = load_and_preprocess_data(zip_file_path="aave_transactions.zip", json_file_name="user-wallet-transactions.json")
    print(f"Loaded {len(df)} transactions.")

    print("Engineering features...")
    wallet_features = engineer_features(df)
    print(f"Engineered features for {len(wallet_features)} unique wallets.")

    print("Assigning credit scores...")
    wallet_scores = assign_credit_scores(wallet_features)

    output_df = wallet_scores.reset_index()
    output_df.columns = ['wallet', 'credit_score']

    output_file = "wallet_scores.csv"
    output_df.to_csv(output_file, index=False)
    print(f"Wallet scores saved to {output_file}")

if __name__ == "__main__":
    main()

Loading and preprocessing data...
Loaded 100000 transactions.
Engineering features...
Engineered features for 3497 unique wallets.
Assigning credit scores...
Wallet scores saved to wallet_scores.csv
